In [27]:
#imbalance
#normalize to 0-1
#out: M,N,D
#some data doesn't exist in B,C and CD-CI, also CV,CW exclude those null instances

import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, nan_euclidean_distances
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD


pd.set_option('display.max_rows', 200)

In [28]:
#create DataFrame
velux_dataset_original = pd.read_csv("sample_data_velux.csv", sep=';', usecols=['if_returning_customer','space_type','project_type','avg_time_per_session_sec','number_of_page_views','duration_of_pages_kidsroom_sec','duration_of_pages_homeoffice_sec','duration_of_pages_livingroom_sec','duration_of_pages_hallwaycorridor_sec','duration_of_pages_kitchen_sec','duration_of_pages_bathroom_sec','duration_of_pages_bedroom_sec','duration_of_pages_extension_sec','duration_of_pages_loftconversion_sec','duration_of_pages_renovation_sec','duration_of_pages_newbuild_sec','duration_of_pages_upgrading_sec','duration_of_pages_reroofing_sec','duration_of_pages_replacement_sec','duration_of_pages_flatroof_sec'])
# 'number_of_pages_allspaces','number_of_pages_kidsroom','number_of_pages_homeoffice','number_of_pages_livingroom','number_of_pages_hallwaycorridor','number_of_pages_kitchen','number_of_pages_bathroom','number_of_pages_bedroom','number_of_pages_extension','number_of_pages_loftconversion','number_of_pages_renovation','number_of_pages_newbuild','number_of_pages_upgrading','number_of_pages_reroofing','number_of_pages_replacement','number_of_pages_flatroof',

In [29]:
#check data type
velux_dataset_original.dtypes

project_type                             object
space_type                               object
if_returning_customer                     int64
avg_time_per_session_sec                  int64
number_of_page_views                      int64
duration_of_pages_kidsroom_sec            int64
duration_of_pages_homeoffice_sec          int64
duration_of_pages_livingroom_sec          int64
duration_of_pages_hallwaycorridor_sec     int64
duration_of_pages_kitchen_sec             int64
duration_of_pages_bathroom_sec            int64
duration_of_pages_bedroom_sec             int64
duration_of_pages_extension_sec           int64
duration_of_pages_loftconversion_sec      int64
duration_of_pages_renovation_sec          int64
duration_of_pages_newbuild_sec            int64
duration_of_pages_upgrading_sec           int64
duration_of_pages_reroofing_sec           int64
duration_of_pages_replacement_sec         int64
duration_of_pages_flatroof_sec            int64
dtype: object

In [30]:
#inspect data
cols = list(velux_dataset_original)
toDelete = []
toObject = []

for col in cols:
    if len(velux_dataset_original[col].unique()) == 1 or (len(velux_dataset_original[col].unique()) == 2 and np.NaN in velux_dataset_original[col].unique()):
         toDelete.append(col)
    elif col[0:2] == 'if':
        toObject.append(col)    

print(toDelete)

[]


In [31]:
#clean the data

#drop cols with only one uniqe value
velux_dataset_original.drop(toDelete, axis='columns',inplace=True)
velux_dataset = velux_dataset_original.copy()
#drop cols with too little data
# velux_dataset = velux_dataset_original.drop(['final_id','first_brproject','first_brspace','marketing_consent_optout_date'], axis='columns')

#convert type of data from int64 to Object where apropriate
for col in toObject:
    velux_dataset[col] = velux_dataset[col].astype(str)


In [32]:
# #create small dataset by deleting all rows with any NaN values
# velux_dataset_small = velux_dataset.dropna()

# #create dataset with synthesied missing values
# velux_dataset_synthesised= [] #TODO

#create big dataset by deleting cols with too many missing data
# velux_dataset_big = velux_dataset.drop(['most_frequent_page_project_type','most_frequent_page_space_type','second_frequent_page_project_type','second_frequent_page_space_type','third_frequent_page_project_type','third_frequent_page_space_type'], axis='columns')
velux_dataset_big = velux_dataset.copy()

In [33]:
velux_dataset_big

,project_type,space_type,if_returning_customer,avg_time_per_session_sec,number_of_page_views,duration_of_pages_kidsroom_sec,duration_of_pages_homeoffice_sec,duration_of_pages_livingroom_sec,duration_of_pages_hallwaycorridor_sec,duration_of_pages_kitchen_sec,duration_of_pages_bathroom_sec,duration_of_pages_bedroom_sec,duration_of_pages_extension_sec,duration_of_pages_loftconversion_sec,duration_of_pages_renovation_sec,duration_of_pages_newbuild_sec,duration_of_pages_upgrading_sec,duration_of_pages_reroofing_sec,duration_of_pages_replacement_sec,duration_of_pages_flatroof_sec
0,replacement,livingroom,1,375,86,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,replacement,bedroom,1,50,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,loftconversion,bedroom,1,3754,144,0,0,0,0,0,0,1122,0,0,6588,0,42,0,22,31981
3,loftconversion,no_space_intent,1,1433,57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,loftconversion,no_space_intent,1,620,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4369,replacement,bedroom,0,167,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4370,replacement,bathroom,0,230,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,153
4371,replacement,bedroom,0,192,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4372,replacement,bathroom,1,206,6,0,0,0,0,0,0,0,0,0,30,0,128,0,0,0


In [34]:
#once again clean cols with only one value
toDeleteBig = []

for col in velux_dataset_big:
    if len(velux_dataset_big[col].unique()) == 1 or (len(velux_dataset_big[col].unique()) == 2 and np.NaN in velux_dataset_big[col].unique()):
        toDeleteBig.append(col)

velux_dataset_big.drop(toDeleteBig,axis='columns',inplace=True)

In [35]:
#check for class imbalance (could use Tomek Links, SMOTE, Metric like F1, and Random Forest algorithm, since tree based algos work well with imbalanced data)
for col in velux_dataset_big:
    print(velux_dataset_big[col].value_counts(), '\n')



project_type
replacement       2720
loftconversion     884
upgrading          427
renovation         180
newbuild           163
Name: count, dtype: int64 

space_type
bedroom                1487
corridororstaircase     605
bathroom                553
livingroom              528
allspaces               437
no_space_intent         283
kitchen                 193
kidsroom                190
office                   98
Name: count, dtype: int64 

if_returning_customer
0    2642
1    1732
Name: count, dtype: int64 

avg_time_per_session_sec
30       33
150      27
90       22
103      21
92       20
         ..
6500      1
13173     1
6145      1
1598      1
507       1
Name: count, Length: 1244, dtype: int64 

number_of_page_views
2      1005
3       748
4       513
5       369
6       273
7       220
8       148
9       141
10      122
11       93
13       87
12       75
14       64
15       63
16       51
19       37
20       33
17       31
18       28
21       22
23       21
27       19

In [36]:
#normalize data
velux_dataset_big_normalized = velux_dataset_big.copy()

for col in velux_dataset_big:
    if velux_dataset_big[col].dtype == 'int64':
        x = velux_dataset_big[col].values
        normalized=(x-x.min())/(x.max()-x.min())
        velux_dataset_big_normalized[col] = normalized


In [68]:
velux_dataset_big['project_type'].unique()

def integerEncoding(x):
    if x == 'replacement':
        return 0
    elif x == 'loftconversion':
        return 1
    elif x == 'upgrading':
        return 2
    elif x == 'newbuild':
        return 3
    elif x == 'renovation':
        return 4

In [83]:
#train classifier and check performance

y = velux_dataset_big['project_type'].apply(integerEncoding)
X = pd.get_dummies(velux_dataset_big.drop('project_type', axis='columns'))
X=X*1.00 #bool --> float64
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45) # make a train-test split to train the model properly

model = Sequential()
model.add(Dense(units=32, activation='sigmoid', input_dim = X_train.shape[1]))
model.add(Dense(units=64, activation='swish'))
model.add(Dense(units=64, activation='swish'))
# model.add(Dropout(rate=0.2))
model.add(Dense(units=64, activation='swish'))
model.add(Dense(units=64, activation='swish'))
model.add(Dense(units=5, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [84]:
model.fit(X_train, y_train, epochs=500, batch_size=32)

Epoch 1/500
110/110 [==============================] - 1s 2ms/step - loss: 1.1462 - accuracy: 0.6156
Epoch 2/500
110/110 [==============================] - 0s 2ms/step - loss: 1.1002 - accuracy: 0.6205
Epoch 3/500
110/110 [==============================] - 0s 2ms/step - loss: 1.0974 - accuracy: 0.6205
Epoch 4/500
110/110 [==============================] - 0s 2ms/step - loss: 1.0945 - accuracy: 0.6202
Epoch 5/500
110/110 [==============================] - 0s 2ms/step - loss: 1.0852 - accuracy: 0.6205
Epoch 6/500
110/110 [==============================] - 0s 2ms/step - loss: 1.0864 - accuracy: 0.6210
Epoch 7/500
110/110 [==============================] - 0s 2ms/step - loss: 1.0831 - accuracy: 0.6239
Epoch 8/500
110/110 [==============================] - 0s 2ms/step - loss: 1.0804 - accuracy: 0.6247
Epoch 9/500
110/110 [==============================] - 0s 2ms/step - loss: 1.0781 - accuracy: 0.6262
Epoch 10/500
110/110 [==============================] - 0s 2ms/step - loss: 1.0761 - accura

In [85]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(test_acc)


28/28 [==============================] - 0s 1ms/step - loss: 1.7069 - accuracy: 0.6206
0.6205714344978333
